In [77]:
import pandas as pd
import numpy as np

In [78]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_train'] = True
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_train'] = False
sample = pd.read_csv("./data/sample_submission.csv")

data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [79]:
data['date'] = pd.to_datetime(data['time'])
data['hour'] = data['date'].dt.hour
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['minute'] = data['date'].dt.minute
data['weekday'] = data['date'].dt.weekday
data['day'] = data['date'].dt.day
data['hour'] = data['date'].dt.hour
# data['ts'] = data['hour']*3600 + data['minute']*60 + data['date'].dt.second
data['ts'] = data['hour']*60 + data['minute']


In [80]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','CS_LL','CS_NH3', 'JS_SW']] #

for f in features:
    data[f] = data[f].fillna(method='ffill')

# 提取特征的diff值

In [81]:
df_list = []
i = 0
add_featuers = []
length = 0
for f in ['JS_NH3', 'CS_TN', 'JS_LL', 'JS_COD', 'CS_COD', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL',
          'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']:
    for r in [15]:
        train_rolling = data[f].rolling(window=r, center=False)
        f_mean_name = 'rolling{}_{}_mean'.format(r,f)
        f_max_name = 'rolling{}_{}_max'.format(r,f)
        f_min_name = 'rolling{}_{}_min'.format(r,f)
        f_std_name = 'rolling{}_{}_std'.format(r,f)
        f_corr_name = 'rolling{}_{}_corr'.format(r,f)
        f_cov_name = 'rolling{}_{}_cov'.format(r,f)
        f_skew_name = 'rolling{}_{}_skew'.format(r,f)
        f_kurt_name = 'rolling{}_{}_kurt'.format(r,f)
        data[f_mean_name] = train_rolling.mean().fillna(0).values
        data[f_max_name] = train_rolling.max().fillna(0).values
        data[f_min_name] = train_rolling.min().fillna(0).values
        data[f_std_name] = train_rolling.std().fillna(0).values
        data[f_corr_name] = train_rolling.corr().fillna(0).values
        data[f_skew_name] = train_rolling.skew().fillna(0).values

        data[f'{f}_{f_mean_name}_cha'] = data[f] - data[f_mean_name]
        data[f'{f}_{f_max_name}_cha'] = data[f] - data[f_max_name]
        data[f'{f}_{f_min_name}_cha'] = data[f] - data[f_min_name]

        data[f'{f}_{f_min_name}_radio'] = data[f'{f}_{f_mean_name}_cha']/data[f]
        data[f'{f}_{f_max_name}_radio'] = data[f'{f}_{f_max_name}_cha']/data[f]
        data[f'{f}_{f_mean_name}_radio'] = data[f'{f}_{f_min_name}_cha']/data[f]
        data[f'{f}_diff'] = data[f].diff()

        if i == 0:
            add_featuers.append(f_mean_name)
            add_featuers.append(f_max_name)
            add_featuers.append(f_min_name)
            add_featuers.append(f_std_name)
            add_featuers.append(f_corr_name)
            add_featuers.append(f_skew_name)
            add_featuers.append(f'{f}_{f_mean_name}_cha')
            add_featuers.append(f'{f}_{f_max_name}_cha')
            add_featuers.append(f'{f}_{f_min_name}_cha')
            add_featuers.append(f'{f}_diff')

features.extend(add_featuers)


gc.collect()
# 对所有的特征进行划分
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())

        data[f] = pd.qcut(data[f], q=int(q/10), labels=False, duplicates="drop")

all_train = data[data['is_train']].reset_index(drop=True)
test = data[~data['is_train']].reset_index(drop=True)

C:\Users\38125\AppData\Local\Temp/ipykernel_51388/3323126436.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_{f_mean_name}_radio'] = data[f'{f}_{f_min_name}_cha']/data[f]
C:\Users\38125\AppData\Local\Temp/ipykernel_51388/3323126436.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{f}_diff'] = data[f].diff()
C:\Users\38125\AppData\Local\Temp/ipykernel_51388/3323126436.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has 

# 删除分布不均衡的特征

In [82]:
all_train = all_train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
def transform(x: pd.Series, c=20):
    return np.log1p(x/c)

def inverse_transform(x: pd.Series, c = 20):
    return np.expm1(x)*c

all_train = all_train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
all_train['Label1'] = transform(all_train['Label1'])
label_c = 8
all_train['Label2'] = transform(all_train['Label2'], c=label_c)

In [83]:
all_train['Label1'].describe()

count    35068.000000
mean         6.489467
std          0.287052
min          3.731088
25%          6.265149
50%          6.484011
75%          6.645066
max          8.106907
Name: Label1, dtype: float64

In [84]:
import matplotlib.pyplot as plt
def plot(y1, y2, x_title, y_title, title):
    plt.figure(figsize=(16,8))
    plt.plot(range(len(y1)), y1)
    plt.plot(range(len(y1)), y2)
    plt.legend()
    plt.xlabel(x_title)
    plt.ylabel(y_title)
    plt.title(title)
    plt.show()

In [89]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
labels = ['Label1','Label2']
score_list = []
test_size = 4000


for label in labels:

    if label == 'Label11':
        use_features = [f for f in features if features.count('diff') == 0] #
        # use_features = features
    else:
        use_features =  features
    X_train = all_train[use_features][:-test_size]
    y_train = all_train[label][:-test_size]

    X_test = all_train[use_features][-test_size:]
    y_test = all_train[label][-test_size:]
    if label == "Label1":
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.03,
            n_estimators=5000,
            subsample=0.4,
            subsample_freq=30,
            min_data_in_leaf=4,
            feature_fraction=0.3,
            bagging_seed=1221,
            reg_alpha=1.5,
            reg_lambda=1.,  # 此处不改了
            min_sum_hessian_in_leaf=1e-1,
            random_state=1212
        )
        # x_copy = X_train[y_train < 9].copy(deep=True)
        # y_copy = y_train[y_train < 9].copy(deep=True)
        # #
        # X_train = np.vstack([x_copy])
        # y_train = np.hstack([y_copy])
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['rmse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)

        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test), inverse_transform(test_pred))))
        # plot(inverse_transform(y_test), inverse_transform(test_pred), "test", 'pred', 'dd')
        print(f"score_list = {score_list}")

        model2 = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.03,
            n_estimators=int(1.2*model.best_iteration_),
            subsample=0.4,
            subsample_freq=30,
            min_data_in_leaf=4,
            feature_fraction=0.3,
            bagging_seed=1221,
            reg_alpha=1.5,
            reg_lambda=1.,  # 此处不改了
            min_sum_hessian_in_leaf=1e-1,
            random_state=1212
        )
        model2.fit(all_train[features], all_train[label])
        test[label] = inverse_transform(model2.predict(test[features]))
    else:
        model = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.03,
            n_estimators=5000,
            min_child_weight=1,
            min_data_in_leaf=40,
            feature_fraction=0.2,
            subsample=0.7,
            subsample_freq=20,
            bagging_seed=1212,
            reg_alpha=0.51,
            reg_lambda=2.5,  # 此处不改了
            min_sum_hessian_in_leaf=1e-1,
            random_state=222
        )
        # x_copy = X_train[y_train > 8.5].copy(deep=True)
        # y_copy = y_train[y_train > 8.5].copy(deep=True)
        #
        # X_train = np.vstack([X_train, x_copy])
        # y_train = np.hstack([y_train, y_copy])
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['rmse'],
              early_stopping_rounds=150, verbose=200)
        test_pred = model.predict(X_test)
        score_list.append(np.sqrt(mean_squared_error(inverse_transform(y_test, c=label_c), inverse_transform(test_pred, c=label_c))))
        print(f"score_list = {score_list}")
        # plot(inverse_transform(y_test), inverse_transform(test_pred), "test", 'pred', 'dd')
        model2 = lgb.LGBMRegressor(
            boosting="gbdt",
            max_depth=4,
            learning_rate=0.03,
            n_estimators=int(1.2*model.best_iteration_),
            min_child_weight=1,
            min_data_in_leaf=40,
            feature_fraction=0.2,
            subsample=0.7,
            subsample_freq=20,
            bagging_seed=1212,
            reg_alpha=0.51,
            reg_lambda=2.5,  # 此处不改了
            min_sum_hessian_in_leaf=1e-1,
            random_state=222
        )
        model2.fit(all_train[features], all_train[label])

        test[label] = inverse_transform(model2.predict(test[features]), c=label_c)

loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.3, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3
[LightGBM] [Warning] min_data_in_leaf is set=4, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=4
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=0.1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=0.1
Training until validation scores don't improve for 150 rounds
[200]	training's rmse: 0.100996	training's l2: 0.0102001	valid_1's rmse: 0.105198	valid_1's l2: 0.0110666
[400]	training's rmse: 0.0883837	training's l2: 0.00781167	valid_1's rmse: 0.101489	valid_1's l2: 0.0103
[600]	training's rmse: 0.0812524	training's l2: 0.00660195	valid_1's rmse: 0.0994868	valid_1's l2: 0.00989762
[800]	training's rmse: 0.0753201	training's l2: 0.00567312	valid_1's rmse: 0.0988837	valid_1's l2: 0.00977798
[1000]	training

In [90]:
loss = np.mean(score_list)
score = 1000/(1+loss)
print(score)

0.8619975030320539


In [91]:
test[['time'] + labels].to_csv(f"./res/lightgbm_random_res.csv", index=False)

In [88]:
score_list

[1054.13499501438]